# Week 03: Classical Machine Learning

Implementing classic ML algorithms from scratch.

## Learning Objectives
1. Implement linear/logistic regression
2. Build decision trees
3. Understand ensemble methods

In [ ]:
import numpy as np
from typing import Tuple, List

## 1. Linear Regression

In [ ]:
class LinearRegression:
    """
    Linear Regression using Normal Equation and Gradient Descent.
    
    Model: y = X @ w + b
    Loss: MSE = (1/n) * Σ(y - ŷ)²
    """
    def __init__(self, method: str = 'normal'):
        self.method = method
        self.weights = None
        self.bias = None
    
    def fit_normal_equation(self, X: np.ndarray, y: np.ndarray):
        X_b = np.c_[np.ones(X.shape[0]), X]
        theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        self.bias = theta[0]
        self.weights = theta[1:]
    
    def fit_gradient_descent(self, X: np.ndarray, y: np.ndarray, lr: float = 0.01, epochs: int = 1000):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0.0
        for _ in range(epochs):
            y_pred = X @ self.weights + self.bias
            error = y - y_pred
            self.weights += lr * (2/n_samples) * X.T @ error
            self.bias += lr * (2/n_samples) * np.sum(error)
    
    def fit(self, X: np.ndarray, y: np.ndarray, **kwargs):
        if self.method == 'normal':
            self.fit_normal_equation(X, y)
        else:
            self.fit_gradient_descent(X, y, **kwargs)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return X @ self.weights + self.bias
    
    def score(self, X: np.ndarray, y: np.ndarray) -> float:
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - ss_res / ss_tot

In [ ]:
# Test Linear Regression
np.random.seed(42)
X = np.random.randn(100, 3)
true_weights = np.array([2, -1, 0.5])
y = X @ true_weights + 3 + np.random.randn(100) * 0.1

model = LinearRegression(method='normal')
model.fit(X, y)
print(f"Learned weights: {model.weights}")
print(f"Learned bias: {model.bias:.4f}")
print(f"R² score: {model.score(X, y):.4f}")

## 2. Logistic Regression

In [ ]:
class LogisticRegression:
    """
    Logistic Regression for binary classification.
    Model: p(y=1|x) = σ(x @ w + b)
    """
    def __init__(self, lr: float = 0.01, epochs: int = 1000):
        self.lr = lr
        self.epochs = epochs
    
    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0.0
        
        for epoch in range(self.epochs):
            z = X @ self.weights + self.bias
            y_pred = self.sigmoid(z)
            
            error = y_pred - y
            dw = (1/n_samples) * X.T @ error
            db = (1/n_samples) * np.sum(error)
            
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        return self.sigmoid(X @ self.weights + self.bias)
    
    def predict(self, X: np.ndarray, threshold: float = 0.5) -> np.ndarray:
        return (self.predict_proba(X) >= threshold).astype(int)
    
    def accuracy(self, X: np.ndarray, y: np.ndarray) -> float:
        return np.mean(self.predict(X) == y)

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=200, n_features=5, random_state=42)

model = LogisticRegression(lr=0.1, epochs=500)
model.fit(X, y)
print(f"Accuracy: {model.accuracy(X, y):.2%}")

## 3. Decision Trees

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTree:
    def __init__(self, max_depth: int = 10, min_samples_split: int = 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    def gini(self, y: np.ndarray) -> float:
        if len(y) == 0: return 0
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)
    
    def information_gain(self, y: np.ndarray, left_idx: np.ndarray, right_idx: np.ndarray) -> float:
        n = len(y)
        if n == 0: return 0
        n_left = len(left_idx)
        n_right = len(right_idx)
        return self.gini(y) - ((n_left/n)*self.gini(y[left_idx]) + (n_right/n)*self.gini(y[right_idx]))
    
    def best_split(self, X: np.ndarray, y: np.ndarray) -> Tuple[int, float]:
        best_gain = -1
        best_feature = None
        best_threshold = None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_idx = np.where(X[:, feature] <= threshold)[0]
                right_idx = np.where(X[:, feature] > threshold)[0]
                if len(left_idx) == 0 or len(right_idx) == 0: continue
                gain = self.information_gain(y, left_idx, right_idx)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold
    
    def build_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        n_samples = len(y)
        n_classes = len(np.unique(y))
        if depth >= self.max_depth or n_classes == 1 or n_samples < self.min_samples_split:
            return Node(value=np.argmax(np.bincount(y)))
        feature, threshold = self.best_split(X, y)
        if feature is None:
            return Node(value=np.argmax(np.bincount(y)))
        left_idx = np.where(X[:, feature] <= threshold)[0]
        right_idx = np.where(X[:, feature] > threshold)[0]
        left_child = self.build_tree(X[left_idx], y[left_idx], depth + 1)
        right_child = self.build_tree(X[right_idx], y[right_idx], depth + 1)
        return Node(feature=feature, threshold=threshold, left=left_child, right=right_child)
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        self.root = self.build_tree(X, y)
    
    def predict_sample(self, x: np.ndarray, node: Node) -> int:
        if node.value is not None: return node.value
        if x[node.feature] <= node.threshold: return self.predict_sample(x, node.left)
        return self.predict_sample(x, node.right)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self.predict_sample(x, self.root) for x in X])

In [ ]:
# Test Decision Tree
X, y = make_classification(n_samples=200, n_features=5, random_state=42)
tree = DecisionTree(max_depth=5)
tree.fit(X, y)
accuracy = np.mean(tree.predict(X) == y)
print(f"Decision Tree Accuracy: {accuracy:.2%}")

## 4. Random Forest & KNN

In [ ]:
class RandomForest:
    def __init__(self, n_estimators: int = 10, max_depth: int = 10):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []
        self.feature_indices = []
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        n_samples, n_features = X.shape
        n_selected = int(np.sqrt(n_features))
        for _ in range(self.n_estimators):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            feat_idx = np.random.choice(n_features, n_selected, replace=False)
            X_sub = X[indices][:, feat_idx]
            y_sub = y[indices]
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_sub, y_sub)
            self.trees.append(tree)
            self.feature_indices.append(feat_idx)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        preds = np.zeros((len(X), self.n_estimators))
        for i, (tree, idx) in enumerate(zip(self.trees, self.feature_indices)):
            preds[:, i] = tree.predict(X[:, idx])
        return np.array([np.argmax(np.bincount(row.astype(int))) for row in preds])

class KNN:
    def __init__(self, k: int = 5):
        self.k = k
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        preds = []
        for x in X:
            dists = [np.sqrt(np.sum((x - xt)**2)) for xt in self.X_train]
            k_idx = np.argsort(dists)[:self.k]
            k_labels = self.y_train[k_idx]
            preds.append(np.argmax(np.bincount(k_labels)))
        return np.array(preds)

In [ ]:
# Test RF and KNN
rf = RandomForest(n_estimators=10, max_depth=5)
rf.fit(X, y)
print(f"Random Forest Accuracy: {np.mean(rf.predict(X) == y):.2%}")

knn = KNN(k=5)
knn.fit(X, y)
print(f"KNN Accuracy: {np.mean(knn.predict(X) == y):.2%}")